In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import scipy as sp
from pylab import text
from soccerplots.radar_chart import Radar
import highlight_text
from functools import reduce

In [ ]:
Competition = 'EPL'
myfont = {'fontname':'DejaVu Sans'}

In [ ]:
filenames = [f"{Competition} Player Stats 2017 - 2018.xlsx",f"{Competition} Player Stats 2018 - 2019.xlsx",f"{Competition} Player Stats 2019 - 2020.xlsx",f"{Competition} Player Stats 2020 - 2021.xlsx"]

In [ ]:
dataframe_list = [pd.read_excel(file_name) for file_name in filenames]

In [ ]:
df0 = dataframe_list[0]
df1 = dataframe_list[1]
df2 = dataframe_list[2]
df3 = dataframe_list[3]

In [ ]:
dataframes = [df0, df1, df2, df3]

In [ ]:
df = pd.concat([df0, df1, df2, df3], ignore_index=True)

In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)
df.to_excel(f"{Competition} Player Stats All Time.xlsx")

In [ ]:
df['Yellow Cards'] = -1*df['Yellow Cards']
df['Red Cards'] = -1*df['Red Cards']
df['Fouls Committed'] = -1*df['Fouls Committed']
df['Turnovers'] = -1*df['Turnovers']

In [ ]:
# PLAYER, POSITION, SEASON and COLOR Selection ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Player_ID_1 = 'Olivier Giroud_Chelsea_2017 - 2018'
Player_1 = str(Player_ID_1).split('_',2)[0]
Team_1 = str(Player_ID_1.split('_',2)[1])
Season_1 = str(Player_ID_1.split('_',2)[2])

Player_ID_2 = 'Olivier Giroud_Arsenal_2017 - 2018'
Player_2 = str(Player_ID_2).split('_',2)[0]
Team_2 = str(Player_ID_2.split('_',2)[1])
Season_2 = str(Player_ID_2.split('_',2)[2])

Color_1 = 'dodgerblue'
Color_2 = 'red'
Comp_1 = 'EPL'
Comp_2 = 'EPL'

Position = 'FW' 

In [ ]:
# FILTERS -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

df = df[df['Position'] != 'GK'] # drop GKs
df = df[df['90s Played'] > 3].reset_index(drop=True) # drop players with less than X 90s Played

df = df[df['Position'] == Position].reset_index(drop='True') # Filter by Single Position

#df = df[(df['Position'] == 'FW') | (df['Position'] == 'FWMF')].reset_index(drop='True') # Filter by Multiple Positions

In [ ]:
# METRICS 1 Select 6 and exclude PLAYER and TEAM ----------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Goalkeepers

#metrics = ['GK Saves','GK Save %','GK Post-Shot xG Against per Shot on Target','GK Post-Shot xG Against PlusMinus', 'GK Long Passes Completed', 'GK Long Pass Preference %']

#Centerbacks

#metrics = ['Pass Completion %', 'Pressures','Fouls Committed','Tackle Success %','Tackles Won','Interceptions']

#Fullbacks

#metrics = ['Tackles + Interceptions','Pressures', 'Deep Progressions','Share of Touches in the Final Third %','Pass Completion %','Goal Contributions per 90']

#Midfielders

#metrics = ['Passes Completed','Pass Completion %','Deep Progressions','xA', 'Goal Contributions per 90', 'Dribbles Completed']

#Attacking Midfielders/Wingers

#metrics = ['Non-Penalty xG','Shots', 'Touches In Opposition Box', 'Pass Completion %', 'Crosses Completed into Opposition Box', 'xA']

#Strikers

metrics = ['Non-Penalty xG','Goals - xG','Shots','Touches In Opposition Box','Shots per Touch', 'xA']

In [ ]:
PPR_1 = []
PPR_2 = []

In [ ]:
Pos_Avg = {"Player": 'Players Combined', "Nationality": 'Players Combined', "Position": 'Players Combined', "Team": 'Players Combined',"Season":'Players Combined',"Player_ID":'Players Combined'}
Pos_Avg = pd.Series(Pos_Avg)
Column_Pos_Averages = df.mean(axis=0)
Pos_Avg =  Pos_Avg.append(Column_Pos_Averages)
Pos_Avg = pd.Series.transpose(Pos_Avg)
df = df.append(Pos_Avg, ignore_index=True)

In [ ]:
fig,axes = plt.subplots(3,2,figsize=(14,10))
fig.set_facecolor('#121212')

mpl.rcParams['xtick.color'] = 'white'
mpl.rcParams['ytick.color'] = 'white'

counter=0
counter2=0 
met_counter = 0

for i,ax in zip(df['Player'],axes.flatten()):
    ax.set_facecolor('#121212')
    ax.grid(ls='dotted',lw=.5,color='lightgrey',axis='y',zorder=1)

    spines = ['top','bottom','left','right']
    for x in spines:
        if x in spines:
            ax.spines[x].set_visible(False)
    
    ax.tick_params(axis=u'both', which=u'both',length=0, color = 'white')

    #Graph 1
    #ax = df[metrics[met_counter]].plot.kde(ax=axes[counter,counter2], zorder=2,color='white')
    #l1 = ax.lines[0]
    #x1 = l1.get_xydata()[:,0]
    #y1 = l1.get_xydata()[:,1]
    #ax.fill_between(x1,y1, color="#7F7F7F", alpha=0.3)

    #Graph 2
    sns.swarmplot(x=metrics[met_counter],data=df,ax=axes[counter,counter2],zorder=2,color='#7F7F7F')
    
    #Graph 3
    #sns.violinplot(x=metrics[met_counter],data=df,ax=axes[counter,counter2],inner = None, zorder=2,color='#7F7F7F')

    per_50 = np.percentile(df[metrics[met_counter]],50)
    ax.axvline(x=per_50,ymin=0.05, ymax=0.95,color='white',zorder=1, linewidth=1.5, ls='-', alpha = 0.5)
    #ax.text(s='-Average',x=per_50,y=-0.43675,c='white',weight='bold', size=8, alpha = 1)

    ax.set_xlabel(f'{metrics[met_counter]}',c='white',size = 12, fontweight = 'bold')

    for x in range(len(df['Player_ID'])):
        if df['Player_ID'][x] == Player_ID_1:
            ax.scatter(x=df[metrics[met_counter]][x],y=0,s=200,c=Color_1,zorder=3)
            #ax.text(s= str(Player_1),x=df[metrics[met_counter]][x],y=-.08,c=Color_1,size=8)
            
            per_player1 = sp.stats.percentileofscore(df[metrics[met_counter]],df[metrics[met_counter]][x])
            PPR_1.append(per_player1)
            text(0.94, 1, str(round(df[metrics[met_counter]][x],2)) + ' - PPR: '+ str(round(per_player1,1)), ha='left', va='center', color=Color_1, transform=ax.transAxes, weight='bold', size=10)

        if df['Player_ID'][x] == Player_ID_2:
            ax.scatter(x=df[metrics[met_counter]][x],y=0,s=200,c=Color_2,zorder=3)
            #ax.text(s= str(Player_2), x=df[metrics[met_counter]][x],y=-.08,c=Color_2,size=8)

            per_player2 = sp.stats.percentileofscore(df[metrics[met_counter]],df[metrics[met_counter]][x])
            PPR_2.append(per_player2)
            text(0.94, 0.94, str(round(df[metrics[met_counter]][x],2))+ ' - PPR: '+ str(round(per_player2,1)), ha='left', va='center', color=Color_2, transform=ax.transAxes, weight='bold', size=10)

    met_counter+=1
    if counter2 == 0:
        counter2 = 1
        continue #break out of the current iteration and go to the top, yoksa bire set edip tekrar sıfıra dönecekti
    if counter2 == 1:
        counter2 = 0
        counter+=1

s = f"<{Player_1}> <{Season_1}> vs. <{Player_2}> <{Season_2}> {Competition}"
highlight_text.fig_text(s=s,
                x=.25, y=0.95,
                highlight_weights = ['bold'],
                **myfont,
                fontsize=22,
                #fontfamily = 'Andale Mono',
                color = 'white',
                highlight_colors = [Color_1, Color_1, Color_2, Color_2], #oyuncu adı rengi buraya
                va='center',
                zorder=3
               )

fig.text(.62,.05,"all stats per 90 / Those with less than 3 90s played excluded",fontsize=11, fontfamily='Andale Mono',color='#7F7F7F')
fig.text(.788,.03,"@msuatgunerli data via FBref",fontsize=11, fontfamily='Andale Mono',color='#7F7F7F')

#plt.savefig('test.png',dpi=500,bbox_inches = 'tight',facecolor=background)


In [ ]:
# METRICS 2 Select 6 and exclude PLAYER and TEAM ----------------------------------------------------------------------------------------------------------------------------------------------------------------------

#Goalkeepers

#metrics = ['GK Average Pass Length (Yd)', 'GK Long Goal Kick Preference %','GK Average Goal Kick Length (Yd)', 'GK Sweeping Actions', 'GK Average Sweeping Action Distance From Goal (Yd)', '90s Played']

#Centerbacks

#metrics = ['Aerials Won','Aerial Win %','Short Passes Completed','Long Passes Completed','Goal Contributions per 90','90s Played']

#Fullbacks

#metrics = ['Dribbles Completed', 'Turnovers', 'Aerials Won', 'Crosses Completed into Opposition Box', 'Tackle Success %','90s Played']

#Midfielders

#metrics = ['Fouls Won','Turnovers','Successful Pressures', 'Pressures', 'Tackles + Interceptions','90s Played']

#Attacking Midfielders/Wingers

#metrics = ['Fouls Won','Dribbles Completed','Turnovers', 'Successful Pressures', 'Goals per Shot','90s Played']

#Strikers

metrics = ['Successful Pressures','Pressures','Aerials Won','Dribbles Completed','Non-Penalty xG per Shot','90s Played']

In [ ]:
fig,axes = plt.subplots(3,2,figsize=(14,10))
fig.set_facecolor('#121212')

mpl.rcParams['xtick.color'] = 'white'
mpl.rcParams['ytick.color'] = 'white'

counter=0
counter2=0 
met_counter = 0

for i,ax in zip(df['Player'],axes.flatten()):
    ax.set_facecolor('#121212')
    ax.grid(ls='dotted',lw=.5,color='lightgrey',axis='y',zorder=1)

    spines = ['top','bottom','left','right']
    for x in spines:
        if x in spines:
            ax.spines[x].set_visible(False)
    
    ax.tick_params(axis=u'both', which=u'both',length=0, color = 'white')

    #Graph 1
    #ax = df[metrics[met_counter]].plot.kde(ax=axes[counter,counter2], zorder=2,color='white')
    #l1 = ax.lines[0]
    #x1 = l1.get_xydata()[:,0]
    #y1 = l1.get_xydata()[:,1]
    #ax.fill_between(x1,y1, color="#7F7F7F", alpha=0.3)

    #Graph 2
    sns.swarmplot(x=metrics[met_counter],data=df,ax=axes[counter,counter2],zorder=2,color='#7F7F7F')
    
    #Graph 3
    #sns.violinplot(x=metrics[met_counter],data=df,ax=axes[counter,counter2],inner = None, zorder=2,color='#7F7F7F')

    per_50 = np.percentile(df[metrics[met_counter]],50)
    ax.axvline(x=per_50,ymin=0.05, ymax=0.95,color='white',zorder=1, linewidth=1.5, ls='-', alpha = 0.5)
    #ax.text(s='-Average',x=per_50,y=-0.43675,c='white',weight='bold', size=8, alpha = 1)

    ax.set_xlabel(f'{metrics[met_counter]}',c='white',size = 12, fontweight = 'bold')

    for x in range(len(df['Player_ID'])):
        if df['Player_ID'][x] == Player_ID_1:
            ax.scatter(x=df[metrics[met_counter]][x],y=0,s=200,c=Color_1,zorder=3)
            #ax.text(s= str(Player_1),x=df[metrics[met_counter]][x],y=-.08,c=Color_1,size=8)
            
            per_player1 = sp.stats.percentileofscore(df[metrics[met_counter]],df[metrics[met_counter]][x])
            PPR_1.append(per_player1)
            text(0.94, 1, str(round(df[metrics[met_counter]][x],2)) + ' - PPR: '+ str(round(per_player1,1)), ha='left', va='center', color=Color_1, transform=ax.transAxes, weight='bold', size=10)

        if df['Player_ID'][x] == Player_ID_2:
            ax.scatter(x=df[metrics[met_counter]][x],y=0,s=200,c=Color_2,zorder=3)
            #ax.text(s= str(Player_2), x=df[metrics[met_counter]][x],y=-.08,c=Color_2,size=8)

            per_player2 = sp.stats.percentileofscore(df[metrics[met_counter]],df[metrics[met_counter]][x])
            PPR_2.append(per_player2)
            text(0.94, 0.94, str(round(df[metrics[met_counter]][x],2))+ ' - PPR: '+ str(round(per_player2,1)), ha='left', va='center', color=Color_2, transform=ax.transAxes, weight='bold', size=10)

    met_counter+=1
    if counter2 == 0:
        counter2 = 1
        continue #break out of the current iteration and go to the top, yoksa bire set edip tekrar sıfıra dönecekti
    if counter2 == 1:
        counter2 = 0
        counter+=1

s = f"<{Player_1}> <{Season_1}> vs. <{Player_2}> <{Season_2}> {Competition}"
highlight_text.fig_text(s=s,
                x=.25, y=0.95,
                highlight_weights = ['bold'],
                **myfont,
                fontsize=22,
                #fontfamily = 'Andale Mono',
                color = 'white',
                highlight_colors = [Color_1, Color_1, Color_2, Color_2], #oyuncu adı rengi buraya
                va='center',
                zorder=3
               )

fig.text(.62,.05,"all stats per 90 / Those with less than 3 90s played excluded",fontsize=11, fontfamily='Andale Mono',color='#7F7F7F')
fig.text(.788,.03,"@msuatgunerli data via FBref",fontsize=11, fontfamily='Andale Mono',color='#7F7F7F')

#plt.savefig('test.png',dpi=500,bbox_inches = 'tight',facecolor=background)


In [ ]:
# PARAMETERS - Select 13 and include PLAYER and TEAM ----------------------------------------------------------------------------------------------------------------------------------------------------------------

#Goalkeepers

#df = df[['Player_ID','Team','GK Saves','GK Save %','GK Post-Shot xG Against per Shot on Target','GK Post-Shot xG Against PlusMinus', 'GK Long Passes Completed', 'GK Long Pass Preference %', 'GK Average Pass Length (Yd)', 'GK Long Goal Kick Preference %','GK Average Goal Kick Length (Yd)', 'GK Sweeping Actions', 'GK Average Sweeping Action Distance From Goal (Yd)']]
#df.columns = ['Player_ID','Team','Saves','Save %','PSxG / SoT','PSxG +/-', 'Long Passes Cmp', 'Long Pass Pref %', 'Avg Pass Length (Yd)', 'Long GK Pref %','Avg GK Length (Yd)', 'Sweeping Actions', 'Avg Sweeping Dist (Yd)']
#Centerbacks

#df = df[['Player_ID','Team','Pass Completion %', 'Pressures','Fouls Committed','Tackle Success %','Tackles Won','Interceptions','Aerials Won','Aerial Win %','Short Passes Completed','Long Passes Completed','Goal Contributions per 90']]
#df.columns = ['Player','Team','Pass Completion %', 'Pressures','Fouls Committed','Tackle Success %','Tackles Won','Interceptions','Aerials Won','Aerial Win %','Short Passes Cmp','Long Passes Cmp','Goal Contributions']

#Fullbacks

#df = df[['Player_ID','Team','Tackles + Interceptions','Pressures', 'Deep Progressions','Share of Touches in the Final Third %','Pass Completion %','Goal Contributions per 90','Dribbles Completed', 'Turnovers', 'Aerials Won', 'Crosses Completed into Opposition Box', 'Tackle Success %']]
#df.columns = ['Player_ID', 'Team','Tackles + Interceptions','Pressures', 'Deep Progressions','Touches Final Third %','Pass Completion %','Goal Contributions','Dribbles Completed', 'Turnovers', 'Aerials Won', ' Crosses Cmp into Box', 'Tackle Success %']

#Midfielders

#df = df[['Player_ID','Team', 'Passes Completed','Pass Completion %','Deep Progressions','xA', 'Goal Contributions per 90', 'Dribbles Completed','Fouls Won','Turnovers','Successful Pressures', 'Pressures', 'Tackles + Interceptions']]
#df.columns = ['Player','Team', 'Passes Cmp','Pass Completion %','Deep Progressions','xA', 'Goal Contributions', 'Dribbles Completed','Fouls Won','Turnovers','Successful Pressures', 'Pressures', 'Tackles + Interceptions']

#Attacking Midfielders/Wingers

#df = df[['Player_ID','Team', 'Non-Penalty xG','Shots', 'Touches In Opposition Box', 'Pass Completion %', 'Crosses Completed into Opposition Box', 'xA','Fouls Won','Dribbles Completed','Turnovers', 'Successful Pressures', 'Goals per Shot']]
#df.columns = ['Player_ID','Team', 'npxG','Shots', 'Touches In Opp. Box', 'Pass Completion %', 'Crosses Cmp into Box', 'xA','Fouls Won','Dribbles Completed','Turnovers', 'Successful Pressures', 'Goals per Shot']

#Strikers

df = df[['Player_ID','Team', 'Non-Penalty xG','Goals - xG','Shots','Touches In Opposition Box','Shots per Touch', 'xA','Successful Pressures','Pressures','Aerials Won','Dribbles Completed','Non-Penalty xG per Shot']]
#df.columns = ['Player_ID','Team', 'Non-Penalty xG','Goals - xG','Shots','Touches In Opp. Box','Shots per Touch', 'xA','Successful Pressures','Pressures','Aerials Won','Dribbles Completed','npxG per Shot']

In [ ]:
dfcomp = df

In [ ]:
df = df[(df['Player_ID']==Player_ID_1) | (df['Player_ID']==Player_ID_2)].reset_index(drop=True)
df

In [ ]:
for w in range(len(df['Player_ID'])):
    if df['Player_ID'][w] == Player_ID_1:
        Team_1 = df.iloc[w].values.tolist()
Team_1 = Team_1[1]

for w in range(len(df['Player_ID'])):
    if df['Player_ID'][w] == Player_ID_2:
        Team_2 = df.iloc[w].values.tolist()
Team_2 = Team_2[1]

In [ ]:
df = df.drop(['Team'], axis=1)

In [ ]:
params = list(df.columns)
params = params[1:]

In [ ]:
per_25 = []
per_50 = []
per_75 = []
PPR_1 = PPR_1[:-1]
PPR_2 = PPR_2[:-1]
labels = []
    
for i in range(len(params)):
   
    per_25v = np.percentile(dfcomp[params[i]],25)
    per_25.append(per_25v)

    per_50v = np.percentile(dfcomp[params[i]],50)
    per_50.append(per_50v)

    per_75v = np.percentile(dfcomp[params[i]],75)
    per_75.append(per_75v)

    label = f'{params[i]}'
    labels.append(label)

In [ ]:
#plt.rcdefaults()
fig, ax = plt.subplots(figsize=(5,7.89))

fig.set_facecolor('#121212')
ax.set_facecolor('#121212')

spines = ['top','bottom','left','right']
for x in spines:
    if x in spines:
        ax.spines[x].set_visible(False)

y = labels
y_pos = np.arange(len(labels))
width = 0.25

plt.axvline(0,linewidth=2, color='white', alpha = 1, zorder = 2)
plt.axvline(25,linewidth=1, color='white', alpha = 0.25, zorder = 1)
plt.axvline(50,linewidth=1, color='white', alpha = 0.25, zorder = 1)
plt.axvline(75,linewidth=1, color='white', alpha = 0.25, zorder = 1)
plt.axvline(100,linewidth=2, color='white', alpha = 0.25, zorder = 1)

rects1 = ax.barh(y_pos - width/2, PPR_1, width, color = Color_1, alpha = 0.95, zorder = 2)
for index, value in enumerate(PPR_1):
    plt.text(1, index - width/4, str(round(df[params[index]][1],2)), fontweight='bold', color='white', zorder = 3)

rects2 = ax.barh(y_pos + width/2, PPR_2, width, color = Color_2, alpha = 0.95, zorder = 2)
for index, value in enumerate(PPR_2):
    plt.text(1, index + 3*width/4, str(round(df[params[index]][0],2)), fontweight='bold', color='white', zorder = 3)

ax.tick_params(axis=u'both', which=u'both',length=0)

plt.xlim([0, 100])
plt.xticks([0,25,50,75,100], fontweight='bold')
ax.set_yticks(y_pos)
ax.set_yticklabels(labels,fontweight='bold', size = 10)
ax.invert_yaxis()
ax.set_xlabel('Player Percentile Rank', fontweight='bold', size = 10, color = 'white')

fig.tight_layout()

plt.show()

In [ ]:
stat1 = 'xA'
stat2 = 'Dribbles Completed'

dfscatter = dfcomp[['Player_ID', stat1, stat2]]

statlist1 = dfscatter[stat1]
statlist1_median = np.percentile(dfcomp[stat1],50)

statlist2 = dfscatter[stat2]
statlist2_median = np.percentile(dfcomp[stat2],50)

#----------------------------------------------------------------------------------------------------------------------------------------------------------------
# get player values for scatter
for w in range(len(dfscatter['Player_ID'])):
    if dfscatter['Player_ID'][w] == Player_ID_1:
        statlist1_player1 = dfscatter.iloc[w].values.tolist()

for w in range(len(dfscatter['Player_ID'])):
    if dfscatter['Player_ID'][w] == Player_ID_2:
        statlist2_player2 = dfscatter.iloc[w].values.tolist()
# -------------------------------------------------------------------------------------------------------------------------------------------------------------
scatter, axes = plt.subplots(figsize=(14,10))

spines = ['top','bottom','left','right']
for sp in spines:
    if sp in spines:
        axes.spines[sp].set_color('white')

plt.xlabel(stat1, color='white', size = 12, fontweight = 'bold')
plt.ylabel(stat2, color='white', size = 12, fontweight = 'bold')
plt.xticks(fontweight='bold')
plt.yticks(fontweight='bold')

axes.axvline(x = statlist1_median, linewidth=1.5, color = '#7F7F7F', zorder=1)
axes.axhline(y = statlist2_median, linewidth=1.5, color = '#7F7F7F', zorder=1)

plt.scatter(x= statlist1_player1[1],y=statlist1_player1[2],s=400,c=Color_1,zorder=3)
plt.text(s= Player_1 + ' ' + Season_1, x= 1.015*statlist1_player1[1],y=1.025*statlist1_player1[2],size=10, fontweight = 'bold', c=Color_1,zorder=3)
plt.scatter(x= statlist2_player2[1],y=statlist2_player2[2],s=400,c=Color_2,zorder=3)
plt.text(s= Player_2 + ' ' + Season_2, x= 1.015*statlist2_player2[1],y=1.025*statlist2_player2[2],size=10, fontweight = 'bold', c=Color_2,zorder=3)

scatter.set_facecolor('#121212')
axes.set_facecolor('#121212')

plt.scatter(statlist1, statlist2, color = 'white', alpha = 0.5, zorder=2)


In [ ]:
ranges = []
a_values = []
b_values = []

for x in params:
    a = min(df[params][x])
    a = a - abs(a*.25)
    
    b = max(df[params][x])
    b = b + abs(b*.10)
    
    ranges.append((a,b))
    
for x in range(len(df['Player_ID'])):
    if df['Player_ID'][x] == Player_ID_1:
        a_values = df.iloc[x].values.tolist()
    if df['Player_ID'][x] == Player_ID_2:
        b_values = df.iloc[x].values.tolist()
        
a_values = a_values[1:]
b_values = b_values[1:]

values = [a_values,b_values]

In [ ]:
title = dict(
    title_name= Player_1,
    title_color = Color_1,
    subtitle_name = str(Team_1) + ' ' + str(Season_1) ,
    subtitle_color = Color_1,
    title_name_2= Player_2,
    title_color_2 = Color_2,
    subtitle_name_2 = str(Team_2) + ' ' + str(Season_2),
    subtitle_color_2 = Color_2,
    title_fontsize = 18,
    subtitle_fontsize = 15
)

endnote = '@msuatgunerli - all stats per90, data via FBref'

In [ ]:
radar = Radar(background_color="#121212", patch_color="#292929", label_color="#FFFFFF", range_color="#FFFFFF")

fig,ax = radar.plot_radar(ranges=ranges,params=params,values=values,
                         radar_color=[Color_1,Color_2],
                         alphas=[0.7,0.6],title=title,endnote=endnote,
                         compare=True)